In [1]:
suppressPackageStartupMessages(library(formattable))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(kableExtra))
suppressPackageStartupMessages(library(RSQLite))

In [3]:
gene_list <- list()
for (i in 1:22) {
    file_name <- paste0("/ysm-gpfs/pi/zhao-data/wl382/cWAS/scripts/zeors_genes_chr", i, ".txt")
    tmp_df <- as.data.frame(fread(file_name)) 
    gene_list[[i]] <- tmp_df 
}

In [8]:
length(unique(gene_list[[1]]$gene))

[1] 202

In [9]:
length(unique(gene_list[[2]]$gene))

[1] 243

In [10]:
length(unique(gene_list[[3]]$gene))

[1] 145

In [12]:
length(unique(gene_list[[4]]$gene))

[1] 220

In [13]:
unique_gene_number <- lapply(gene_list, function(x) length(unique(x$gene)))

In [14]:
unique_gene_number

[[1]]
[1] 202

[[2]]
[1] 243

[[3]]
[1] 145

[[4]]
[1] 220

[[5]]
[1] 212

[[6]]
[1] 132

[[7]]
[1] 117

[[8]]
[1] 192

[[9]]
[1] 117

[[10]]
[1] 109

[[11]]
[1] 191

[[12]]
[1] 127

[[13]]
[1] 97

[[14]]
[1] 136

[[15]]
[1] 101

[[16]]
[1] 115

[[17]]
[1] 66

[[18]]
[1] 92

[[19]]
[1] 99

[[20]]
[1] 69

[[21]]
[1] 86

[[22]]
[1] 41

In [15]:
gene_list[[1]]

gene,chr,tissue
<chr>,<int>,<chr>
ENSG00000000457,1,Brain_Amygdala
ENSG00000000457,1,Brain_Cerebellar_Hemisphere
ENSG00000000457,1,Brain_Cerebellum
ENSG00000000457,1,Brain_Frontal_Cortex_BA9
ENSG00000000457,1,Brain_Spinal_cord_cervical_c-1
ENSG00000000457,1,Brain_Substantia_nigra
ENSG00000000457,1,Cells_Cultured_fibroblasts
ENSG00000000457,1,Colon_Transverse
ENSG00000000457,1,Liver


### check the raw expression

In [17]:

exp_dir <- "/gpfs/loomis/scratch60/fas/radev/zy92/GTEX/expr_gtex1/"
check_raw_expression <- function(gene, chr, exp_dir) {
    gene_dir <- paste0(exp_dir, chr, gene)
    tissue_vec <- list.files(gene_dir)
    exp_list <- list()
    for (tissue in tissue_vec) {
        tissue_file <- paste0(gene_dir, tissue)
        tmp_df <- as.data.frame(fread(tissue_file))
        exp_list[[tissue]] <- tmp_df
    }
    return(exp_list)
    
}

In [20]:
test_list <- check_raw_expression(exp_dir, "chr1/", "ENSG00000000457/")

In [22]:
zero_gene_vec <- unlist(lapply(gene_list, function(x) unique(x$gene)))

In [25]:
length(zero_gene_vec)
length(unique(zero_gene_vec))

[1] 2909

[1] 2909

In [26]:
# grch37 
library(biomaRt)
suppressPackageStartupMessages(library(biomaRt))
grch37 = useMart(biomart="ENSEMBL_MART_ENSEMBL", host="grch37.ensembl.org", path="/biomart/martservice", dataset="hsapiens_gene_ensembl")
gene_info = getBM(filters= "ensembl_gene_id", 
                  attributes= c("ensembl_gene_id",'hgnc_symbol','chromosome_name',"transcript_biotype"), 
                  values= zero_gene_vec, 
                  mart= grch37)



In [27]:
head(gene_info)

ensembl_gene_id,hgnc_symbol,chromosome_name,transcript_biotype
<chr>,<chr>,<int>,<chr>
ENSG00000163655,GMPS,3,protein_coding
ENSG00000163655,GMPS,3,processed_transcript
ENSG00000179674,ARL14,3,protein_coding
ENSG00000169255,B3GALNT1,3,protein_coding
ENSG00000169255,B3GALNT1,3,processed_transcript
ENSG00000169282,KCNAB1,3,retained_intron


In [29]:
table(gene_info$transcript_biotype)


              antisense                 lincRNA                   miRNA 
                      4                    1655                     520 
               misc_RNA nonsense_mediated_decay  polymorphic_pseudogene 
                      1                     163                       1 
   processed_transcript          protein_coding         retained_intron 
                    337                     744                     272 
                   rRNA                  snoRNA                   snRNA 
                      2                       3                       1 

In [21]:
test_list

list()

In [ ]:
raw_expression_list <- list()
raw_expression_dir <- 
for (i in 1:22) {
    
}

In [ ]:
library(biomart)